In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER

/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER


In [ ]:
!pip install -r req.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch.utils.data as data
from torchvision import transforms
import torch
import os
import argparse
from data_preprocessing.dataset_raf import RafDataSet
from data_preprocessing.dataset_affectnet import Affectdataset
from data_preprocessing.dataset_affectnet_8class import Affectdataset_8class
import random

from utils import *
from models.emotion_hyp import pyramid_trans_expr
from sklearn.metrics import confusion_matrix
from data_preprocessing.plot_confusion_matrix import plot_confusion_matrix

import cv2


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = str('0,1')
print("Work on GPU: ", os.environ['CUDA_VISIBLE_DEVICES'])

Work on GPU:  0,1


In [ ]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
num_classes = 7
model = pyramid_trans_expr(img_size=224, num_classes=num_classes, type="large")

load_weight 304


In [ ]:
sample = cv2.imread("/content/drive/MyDrive/NLP_Project/test_0001_aligned.jpg")

In [ ]:
#sample = sample[:, :, ::-1]  # BGR to RGB

In [ ]:
def add_gaussian_noise(image_array, mean=0.0, var=30):
    std = var**0.5
    noisy_img = image_array + np.random.normal(mean, std, image_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img_clipped

def flip_image(image_array):
    return cv2.flip(image_array, 1)

In [ ]:
aug_func = [flip_image, add_gaussian_noise]

In [ ]:
index = random.randint(0, 1)
sample = aug_func[index](sample)

In [ ]:
sample = transform(sample.copy())

In [ ]:
print("Loading pretrained weights...", "/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = torch.load("/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = checkpoint["model_state_dict"]
model = load_pretrained_weights(model, checkpoint)

Loading pretrained weights... /content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth
load_weight 1309


In [ ]:
model = model.cuda()

In [ ]:
model

pyramid_trans_expr(
  (face_landback): MobileFaceNet(
    (conv1): Conv_block(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (conv2_dw): Conv_block(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=64)
    )
    (conv_23): Depth_Wise(
      (conv): Conv_block(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (prelu): PReLU(num_parameters=128)
      )
      (conv_dw): Conv_block(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
        

In [ ]:
type(sample)

torch.Tensor

In [ ]:
sample = [sample]

In [ ]:
test_loader = torch.utils.data.DataLoader(sample,
                                             batch_size=32,
                                             num_workers=2,
                                             shuffle=False,
                                             pin_memory=True)

In [ ]:
test_loader

In [ ]:
with torch.no_grad():
    bingo_cnt = 0
    model.eval()
    for batch_i, (imgs) in enumerate(test_loader):
        outputs, features = model(imgs.cuda())
        _, predicts = torch.max(outputs, 1)
        print(predicts)
        print(outputs)

In [ ]:
labels_name = ['Surprise', 'Fear', 'Disgust', 'Happiness', 'Sadness', 'Anger', 'Neutral']
print(labels_name[predicts.item()] + " " + str(predicts.item() + 1))

Sadness 5


In [ ]:
import cv2
import os
import random

In [ ]:

def save_random_frame_from_section(video_path, output_folder, section_start, section_end, section_number):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(section_start * fps)
    end_frame = int(section_end * fps)

    random_frame = random.randint(start_frame, end_frame)
    cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame)

    ret, frame = cap.read()
    if ret:
        frame_filename = os.path.join(output_folder, f"frame_section_{section_number}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved frame from section {section_number} to {frame_filename}")
    else:
        print(f"Failed to capture frame from section {section_number}")

    cap.release()

In [ ]:
def extract_frames(video_path, output_folder, num_sections=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps

    section_duration = duration / 4

    sections_to_extract = random.sample(range(1, 5), min(num_sections, 4))

    for section_number in sections_to_extract:
        section_start = (section_number - 1) * section_duration
        section_end = section_number * section_duration
        save_random_frame_from_section(video_path, output_folder, section_start, section_end, section_number)

    cap.release()

In [ ]:
video_path = "/content/drive/MyDrive/NLP_Project/cropped_outputs/video02.mp4"
output_folder = "/content/drive/MyDrive/NLP_Project/Photos_to_predict"
num_sections = 4

In [ ]:
extract_frames(video_path, output_folder, num_sections)


Saved frame from section 1 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_1.jpg
Saved frame from section 2 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_2.jpg
Saved frame from section 3 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_3.jpg
Saved frame from section 4 to /content/drive/MyDrive/NLP_Project/Photos_to_predict/frame_section_4.jpg


In [ ]:
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        if os.path.isfile(img_path):
            # Read the image
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Could not read image: {img_path}")
    return images

# Provide the path to the folder containing images
folder_path = "/content/drive/MyDrive/NLP_Project/Photos_to_predict"
images_array = read_images_from_folder(folder_path)

# Now you have all the images stored in the `images_array` list

In [ ]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
num_classes = 7
model = pyramid_trans_expr(img_size=224, num_classes=num_classes, type="large")

load_weight 304


In [ ]:
def add_gaussian_noise(image_array, mean=0.0, var=30):
    std = var**0.5
    noisy_img = image_array + np.random.normal(mean, std, image_array.shape)
    noisy_img_clipped = np.clip(noisy_img, 0, 255).astype(np.uint8)
    return noisy_img_clipped

def flip_image(image_array):
    return cv2.flip(image_array, 1)

In [ ]:
aug_func = [flip_image, add_gaussian_noise]

In [ ]:
index = 0
for img in images_array:
  i = random.randint(0, 1)
  img = aug_func[i](img)
  img = transform(img.copy())
  images_array[index] = img
  index += 1

In [ ]:
test_loader = torch.utils.data.DataLoader(images_array,
                                             batch_size=32,
                                             num_workers=2,
                                             shuffle=False,
                                             pin_memory=True)

In [ ]:
print("Loading pretrained weights...", "/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = torch.load("/content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth")
checkpoint = checkpoint["model_state_dict"]
model = load_pretrained_weights(model, checkpoint)

Loading pretrained weights... /content/drive/MyDrive/SentimentalAnalyze/POSTER_AND_LDLVA/POSTER/checkpoint/rafdb_best.pth
load_weight 1309


In [ ]:
model = model.cuda()

In [ ]:
with torch.no_grad():
    bingo_cnt = 0
    model.eval()
    for batch_i, (imgs) in enumerate(test_loader):
        outputs, features = model(imgs.cuda())
        _, predicts = torch.max(outputs, 1)
        print(predicts)
        print(outputs)

tensor([3, 2, 2, 0], device='cuda:0')
tensor([[-0.4358, -1.0675,  0.3217,  2.8434, -0.8518, -0.3195, -0.4349],
        [ 0.2487, -1.2416,  1.9898,  0.5242, -0.4681,  0.1476, -0.6603],
        [ 0.9859, -1.3570,  1.5785,  0.0784, -0.4941,  0.0539, -0.7567],
        [ 1.3812, -1.3101,  1.2315, -0.0459, -0.3129,  0.3867, -1.1606]],
       device='cuda:0')


AttributeError: 'builtin_function_or_method' object has no attribute 'item'

In [ ]:
labels_name = ['Surprise', 'Fear', 'Disgust', 'Happiness', 'Sadness', 'Anger', 'Neutral']
print(labels_name[predicts.item()[0]] + " " + str(predicts.item()[0] + 1))

RuntimeError: a Tensor with 4 elements cannot be converted to Scalar

In [ ]:
import torch
import torch.nn.functional as F


In [ ]:
# Apply softmax along the specified dimension (default is dim=1)
softmax_output = F.softmax(outputs, dim=1)
softmax_output

tensor([[0.0303, 0.0161, 0.0646, 0.8046, 0.0200, 0.0340, 0.0303],
        [0.0996, 0.0224, 0.5680, 0.1312, 0.0486, 0.0900, 0.0401],
        [0.2436, 0.0234, 0.4406, 0.0983, 0.0555, 0.0959, 0.0426],
        [0.3570, 0.0242, 0.3074, 0.0857, 0.0656, 0.1321, 0.0281]],
       device='cuda:0')

In [ ]:

# Convert softmax_output tensor to numpy array
softmax_array = softmax_output.cpu().detach().numpy()


In [ ]:
%cd /content/drive/MyDrive/NLP_Project

/content/drive/MyDrive/NLP_Project


In [ ]:

# Save the softmax arrays to a text file
with open("softmax_output_of_video.txt", "w") as file:
    for array in softmax_array:
        file.write(' '.join([str(elem) for elem in array]) + '\n')